# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv file from Part I WeatherPy and Store in a DataFrame
file = "output_data/Weather_by_City.csv"
weather_data = pd.read_csv(file)
weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Castro,-24.7911,-50.0119,44.04,97,13,2.24,BR,1627879406
1,1,Port Augusta,-32.5000,137.7667,62.71,70,86,18.61,AU,1627879406
2,2,Busselton,-33.6500,115.3333,58.05,66,100,10.36,AU,1627879407
3,3,Karratha,-20.7377,116.8463,76.21,25,0,15.41,AU,1627879408
4,4,Hilo,19.7297,-155.0900,75.79,88,65,6.22,US,1627879004


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Congifure gmaps
gmaps.configure(api_key=g_key)

# Coordinates of the weather data
locations = weather_data[["Lat", "Lng"]]
weight = weather_data["Humidity"]

In [4]:
# Draw the map
figure_layout = {"width": "400px", "height": "300px", "border": "1px solid black", "padding": "1px",
                 "margin": "0 auto 0 auto"}

fig = gmaps.figure(figure_layout, center=(10,0), zoom_level=2)

# Add a heatmap
max_humidity = weather_data["Humidity"].max()
heat_layer = gmaps.heatmap_layer(locations, weight)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# I want to go somewhere with 0-30 % humidity, max temp 85 F, and wind speed less than 7 mph

best_vacation = weather_data.loc[(weather_data["Humidity"] < 30) & (weather_data["Max Temp"] < 85) &
                                (weather_data["Wind Speed"] < 7)] #6 places
best_vacation = best_vacation.dropna(how="any")
best_vacation = best_vacation.reset_index() 
best_vacation # no null values -- 6 rows

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,138,138,Los Andes,-32.8337,-70.5983,59.02,14,0,5.77,CL,1627879545
1,248,248,Ust'-Ilimsk,58.0006,102.6619,78.24,27,57,5.66,RU,1627879670
2,310,310,Esna,25.2934,32.5540,80.62,15,6,5.12,EG,1627879771
3,387,387,Sabzawār,36.2126,57.6819,76.28,27,0,6.91,IR,1627879847
4,431,431,Batagay,67.6333,134.6333,75.63,24,0,2.91,RU,1627879906
5,560,560,Yantal',56.8471,105.2542,77.07,29,6,6.71,RU,1627880070


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store the above DataFrame into hotel_df and drop the index and Unnamed columns
hotel_df = best_vacation.drop(["index", "Unnamed: 0"], axis=1)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Los Andes,-32.8337,-70.5983,59.02,14,0,5.77,CL,1627879545
1,Ust'-Ilimsk,58.0006,102.6619,78.24,27,57,5.66,RU,1627879670
2,Esna,25.2934,32.5540,80.62,15,6,5.12,EG,1627879771
3,Sabzawār,36.2126,57.6819,76.28,27,0,6.91,IR,1627879847
4,Batagay,67.6333,134.6333,75.63,24,0,2.91,RU,1627879906
5,Yantal',56.8471,105.2542,77.07,29,6,6.71,RU,1627880070


In [7]:
# Use Google Places API to find the first hotel for each city located within 5000 meters of your coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "Hotel"
target_radius = 5000
target_type = "Lodging"
        
hotel_list = []

x = 0

for city in hotel_df["City"]:
    target_coordinates = str(hotel_df["Lat"][x]) + ', ' + str(hotel_df["Lng"][x])
    x = x + 1
    
    params={"location":target_coordinates, "keyword":target_search, "radius":target_radius, "type":target_type,
           "key":g_key}
    
    try:
        response = requests.get(base_url, params=params).json()
        hotel_name = response['results'][0]['name']
        hotel_list.append(hotel_name)
    
    except IndexError:
        hotel_list.append(np.nan)
    
hotel_list

['Cardon',
 'Gostinitsa "Ust\'-Ilim"',
 'Orient Star Dahabyia Isna 05 Nights Each Sunday',
 'هتل کاملیا',
 nan,
 nan]

In [8]:
# Add the hotel_list to the hotel_df DataFrame
hotel_df["Hotel Name"] = hotel_list
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Los Andes,-32.8337,-70.5983,59.02,14,0,5.77,CL,1627879545,Cardon
1,Ust'-Ilimsk,58.0006,102.6619,78.24,27,57,5.66,RU,1627879670,"Gostinitsa ""Ust'-Ilim"""
2,Esna,25.2934,32.5540,80.62,15,6,5.12,EG,1627879771,Orient Star Dahabyia Isna 05 Nights Each Sunday
3,Sabzawār,36.2126,57.6819,76.28,27,0,6.91,IR,1627879847,هتل کاملیا
4,Batagay,67.6333,134.6333,75.63,24,0,2.91,RU,1627879906,NaN
5,Yantal',56.8471,105.2542,77.07,29,6,6.71,RU,1627880070,NaN


In [9]:
# Clean the DataFrame by removing NaN values
hotel_df = hotel_df.dropna(how="any")
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Los Andes,-32.8337,-70.5983,59.02,14,0,5.77,CL,1627879545,Cardon
1,Ust'-Ilimsk,58.0006,102.6619,78.24,27,57,5.66,RU,1627879670,"Gostinitsa ""Ust'-Ilim"""
2,Esna,25.2934,32.5540,80.62,15,6,5.12,EG,1627879771,Orient Star Dahabyia Isna 05 Nights Each Sunday
3,Sabzawār,36.2126,57.6819,76.28,27,0,6.91,IR,1627879847,هتل کاملیا


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_coordinates = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(marker_coordinates)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))